# Effective Kraus Rank Analysis

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

C:\Users\krist\Documents\ModelNISQ\notebooks\experiments\../../src_tf\spam.py:111: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if init is "random":
C:\Users\krist\Documents\ModelNISQ\notebooks\experiments\../../src_tf\spam.py:115: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif init is "ideal":
C:\Users\krist\Documents\ModelNISQ\notebooks\experiments\../../src_tf\spam.py:125: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if povm is "random":
C:\Users\krist\Documents\ModelNISQ\notebooks\experiments\../../src_tf\spam.py:133: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif povm is "ideal":


ModuleNotFoundError: No module named 'quantum_maps'

For a quantum channel $\Lambda_a$ defined on a d-dimensional Hilbert space, we introduce two scalar values that characterize the channel:

The Effective Kraus Rank of $\Lambda_a$ is given by  $r_{eff,a} = \frac{d^2}{\tau}$, where $\tau = \sum_{i=1}^{d^2} \lambda_i^2$ is the purity of the channel choi matrix. The effective kraus rank is a statement of the distribution on the eigenvalues on the choi. Since $\sum_{i=1}^{d^2} \lambda_i = d$, $r_{eff} = 1$ tells us that the map has only a single non-zero eigenvalue. 

The Attraction of $\Lambda_a$ is given by $Att_a = \int_{\rho \in U_{Haar}(d)} F(\Lambda_a(\rho), I/d)d\rho$, where $F(A,B) = Tr(\sqrt{\sqrt{A}B\sqrt{A}})^2$ is the state fidelity. The Attraction quantifies to what degree the channel maps pure states, on average, towards the maximally mixed state. 

Conjecture:

For Quantum Channels $\Lambda_a$ and $\Lambda_b$, we have the following equivalence.

$r_{eff,a} \leq r_{eff,b} \Longleftrightarrow Att_a \leq Att_b$

Optimizing counter examples to this conjecture has so far failed.

## Attraction

In [5]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
n = 3
d = 2**n

kraus1= KrausMap(d = d, 
                 rank = d**2)

print(attraction(kraus1))

tf.Tensor(0.9672357814047119, shape=(), dtype=float64)


In [6]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
n = 3
d = 2**n

kraus1= KrausMap(d = d, 
                 rank = d**2)

model1= ModelQuantumMap(
                        q_map = kraus1,
                        loss = AttractionRankTradeoff(inflate=True),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )
model1.train(inputs = [],
            targets = [],
            num_iter = 100,
            N = 1,
            use_batch=False,
            verbose=True,
           )



kraus2= KrausMap(d = d, 
                 rank = d**2)
model2 = ModelQuantumMap(
                        q_map = kraus2,
                        loss = AttractionRankTradeoff(inflate=False),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )
model2.train(inputs = [],
            targets = [],
            num_iter = 100,
            N = 1,
            use_batch=False,
            verbose=True,
           )

  0%|          | 0/100 [00:00<?, ?it/s]

Step:0, train: 0.46488+0.00000j, val: 0.00000
Step:1, train: 0.45797+0.00000j, val: 0.00000
Step:2, train: 0.45089-0.00000j, val: 0.00000
Step:3, train: 0.44386-0.00000j, val: 0.00000
Step:4, train: 0.43684-0.00000j, val: 0.00000
Step:5, train: 0.42991-0.00000j, val: 0.00000
Step:6, train: 0.42285+0.00000j, val: 0.00000
Step:7, train: 0.41588-0.00000j, val: 0.00000
Step:8, train: 0.40892+0.00000j, val: 0.00000
Step:9, train: 0.40210-0.00000j, val: 0.00000
Step:10, train: 0.39508-0.00000j, val: 0.00000
Step:11, train: 0.38828-0.00000j, val: 0.00000
Step:12, train: 0.38142+0.00000j, val: 0.00000
Step:13, train: 0.37457+0.00000j, val: 0.00000
Step:14, train: 0.36787+0.00000j, val: 0.00000
Step:15, train: 0.36115-0.00000j, val: 0.00000
Step:16, train: 0.35447-0.00000j, val: 0.00000
Step:17, train: 0.34786+0.00000j, val: 0.00000
Step:18, train: 0.34132-0.00000j, val: 0.00000
Step:19, train: 0.33481+0.00000j, val: 0.00000
Step:20, train: 0.32834-0.00000j, val: 0.00000
Step:21, train: 0.32185

  0%|          | 0/100 [00:00<?, ?it/s]

Step:0, train: -0.46325-0.00000j, val: 0.00000
Step:1, train: -0.47025-0.00000j, val: 0.00000
Step:2, train: -0.47713+0.00000j, val: 0.00000
Step:3, train: -0.48415+0.00000j, val: 0.00000
Step:4, train: -0.49090-0.00000j, val: 0.00000
Step:5, train: -0.49764-0.00000j, val: 0.00000
Step:6, train: -0.50438+0.00000j, val: 0.00000
Step:7, train: -0.51108-0.00000j, val: 0.00000
Step:8, train: -0.51758+0.00000j, val: 0.00000
Step:9, train: -0.52406-0.00000j, val: 0.00000
Step:10, train: -0.53036-0.00000j, val: 0.00000
Step:11, train: -0.53662-0.00000j, val: 0.00000
Step:12, train: -0.54288+0.00000j, val: 0.00000
Step:13, train: -0.54897-0.00000j, val: 0.00000
Step:14, train: -0.55491-0.00000j, val: 0.00000
Step:15, train: -0.56087-0.00000j, val: 0.00000
Step:16, train: -0.56639-0.00000j, val: 0.00000
Step:17, train: -0.57224-0.00000j, val: 0.00000
Step:18, train: -0.57771+0.00000j, val: 0.00000
Step:19, train: -0.58333-0.00000j, val: 0.00000
Step:20, train: -0.58839+0.00000j, val: 0.00000
St

In [7]:
print(effective_rank(kraus1), effective_rank(kraus2))
print(attraction(kraus1), attraction(kraus2))

tf.Tensor((62.67683617128692-1.0495220191603566e-16j), shape=(), dtype=complex128) tf.Tensor((7.040094984077804+6.167949070231107e-16j), shape=(), dtype=complex128)
tf.Tensor(0.9992603594638745, shape=(), dtype=float64) tf.Tensor(0.7050291621562537, shape=(), dtype=float64)


## Equal effective rank, check attraction

In [16]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
n = 3
d = 2**n

kraus1= KrausMap(d = d, 
                 rank = d**2)

model1= ModelQuantumMap(
                        q_map = kraus1,
                        loss = RankShrink(inflate=True),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )
model1.train(inputs = [],
            targets = [],
            num_iter = 50,
            N = 1,
            use_batch=False,
            verbose=True,
           )



kraus2= KrausMap(d = d, 
                 rank = d**2)
model2 = ModelQuantumMap(
                        q_map = kraus2,
                        loss = RankShrink(),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )
model2.train(inputs = [],
            targets = [],
            num_iter = 100,
            N = 1,
            use_batch=False,
            verbose=True,
           )

  0%|          | 0/50 [00:00<?, ?it/s]

Step:0, train: -32.14314+0.00000j, val: 0.00000
Step:1, train: -32.65734-0.00000j, val: 0.00000
Step:2, train: -33.17094-0.00000j, val: 0.00000
Step:3, train: -33.68334+0.00000j, val: 0.00000
Step:4, train: -34.19409-0.00000j, val: 0.00000
Step:5, train: -34.70268-0.00000j, val: 0.00000
Step:6, train: -35.20885-0.00000j, val: 0.00000
Step:7, train: -35.71245-0.00000j, val: 0.00000
Step:8, train: -36.21340-0.00000j, val: 0.00000
Step:9, train: -36.71136+0.00000j, val: 0.00000
Step:10, train: -37.20588+0.00000j, val: 0.00000
Step:11, train: -37.69687-0.00000j, val: 0.00000
Step:12, train: -38.18384-0.00000j, val: 0.00000
Step:13, train: -38.66654+0.00000j, val: 0.00000
Step:14, train: -39.14459+0.00000j, val: 0.00000
Step:15, train: -39.61855+0.00000j, val: 0.00000
Step:16, train: -40.08821-0.00000j, val: 0.00000
Step:17, train: -40.55429-0.00000j, val: 0.00000
Step:18, train: -41.01611+0.00000j, val: 0.00000
Step:19, train: -41.47372-0.00000j, val: 0.00000
Step:20, train: -41.92701-0.00

  0%|          | 0/100 [00:00<?, ?it/s]

Step:0, train: 32.46070-0.00000j, val: 0.00000
Step:1, train: 31.94841-0.00000j, val: 0.00000
Step:2, train: 31.43722+0.00000j, val: 0.00000
Step:3, train: 30.92755-0.00000j, val: 0.00000
Step:4, train: 30.41976-0.00000j, val: 0.00000
Step:5, train: 29.91436-0.00000j, val: 0.00000
Step:6, train: 29.41168-0.00000j, val: 0.00000
Step:7, train: 28.91191-0.00000j, val: 0.00000
Step:8, train: 28.41559-0.00000j, val: 0.00000
Step:9, train: 27.92291-0.00000j, val: 0.00000
Step:10, train: 27.43420+0.00000j, val: 0.00000
Step:11, train: 26.94957+0.00000j, val: 0.00000
Step:12, train: 26.46921+0.00000j, val: 0.00000
Step:13, train: 25.99290-0.00000j, val: 0.00000
Step:14, train: 25.52096+0.00000j, val: 0.00000
Step:15, train: 25.05355+0.00000j, val: 0.00000
Step:16, train: 24.59107-0.00000j, val: 0.00000
Step:17, train: 24.13349+0.00000j, val: 0.00000
Step:18, train: 23.68091-0.00000j, val: 0.00000
Step:19, train: 23.23329+0.00000j, val: 0.00000
Step:20, train: 22.79146+0.00000j, val: 0.00000
St

In [17]:
model1.loss = RankMSE()
model1.zero_optimizer()

model2.loss = RankMSE()
model2.zero_optimizer()

model1.train(inputs = 8,
            targets = 8,
            num_iter = 1000,
            N = 1,
            use_batch=False,
            verbose=True,
           )

model2.train(inputs = 8,
            targets = 8,
            num_iter = 1000,
            N = 1,
            use_batch=False,
            verbose=True,
           )

  0%|          | 0/1000 [00:00<?, ?it/s]

Step:0, train: 2092.62379-0.00000j, val: 0.00000
Step:1, train: 2060.54552-0.00000j, val: 0.00000
Step:2, train: 2027.93866+0.00000j, val: 0.00000
Step:3, train: 1994.87996-0.00000j, val: 0.00000
Step:4, train: 1961.43149+0.00000j, val: 0.00000
Step:5, train: 1927.63664+0.00000j, val: 0.00000
Step:6, train: 1893.53590-0.00000j, val: 0.00000
Step:7, train: 1859.15188-0.00000j, val: 0.00000
Step:8, train: 1824.53593-0.00000j, val: 0.00000
Step:9, train: 1789.71310+0.00000j, val: 0.00000
Step:10, train: 1754.71729+0.00000j, val: 0.00000
Step:11, train: 1719.56259-0.00000j, val: 0.00000
Step:12, train: 1684.28669+0.00000j, val: 0.00000
Step:13, train: 1648.90699+0.00000j, val: 0.00000
Step:14, train: 1613.45438-0.00000j, val: 0.00000
Step:15, train: 1577.90156+0.00000j, val: 0.00000
Step:16, train: 1542.29097-0.00000j, val: 0.00000
Step:17, train: 1506.62347-0.00000j, val: 0.00000
Step:18, train: 1470.89236+0.00000j, val: 0.00000
Step:19, train: 1435.10472-0.00000j, val: 0.00000
Step:20, t

Step:168, train: 7.68635+0.00000j, val: 0.00000
Step:169, train: 7.50677-0.00000j, val: 0.00000
Step:170, train: 7.33199+0.00000j, val: 0.00000
Step:171, train: 7.16186+0.00000j, val: 0.00000
Step:172, train: 6.99623-0.00000j, val: 0.00000
Step:173, train: 6.83496-0.00000j, val: 0.00000
Step:174, train: 6.67792+0.00000j, val: 0.00000
Step:175, train: 6.52498-0.00000j, val: 0.00000
Step:176, train: 6.37600-0.00000j, val: 0.00000
Step:177, train: 6.23088-0.00000j, val: 0.00000
Step:178, train: 6.08948-0.00000j, val: 0.00000
Step:179, train: 5.95170+0.00000j, val: 0.00000
Step:180, train: 5.81744-0.00000j, val: 0.00000
Step:181, train: 5.68658+0.00000j, val: 0.00000
Step:182, train: 5.55902+0.00000j, val: 0.00000
Step:183, train: 5.43467-0.00000j, val: 0.00000
Step:184, train: 5.31343+0.00000j, val: 0.00000
Step:185, train: 5.19522+0.00000j, val: 0.00000
Step:186, train: 5.07994-0.00000j, val: 0.00000
Step:187, train: 4.96751+0.00000j, val: 0.00000
Step:188, train: 4.85785-0.00000j, val: 

Step:340, train: 0.21422-0.00000j, val: 0.00000
Step:341, train: 0.20991+0.00000j, val: 0.00000
Step:342, train: 0.20568-0.00000j, val: 0.00000
Step:343, train: 0.20154+0.00000j, val: 0.00000
Step:344, train: 0.19748-0.00000j, val: 0.00000
Step:345, train: 0.19350+0.00000j, val: 0.00000
Step:346, train: 0.18960+0.00000j, val: 0.00000
Step:347, train: 0.18578-0.00000j, val: 0.00000
Step:348, train: 0.18203+0.00000j, val: 0.00000
Step:349, train: 0.17835-0.00000j, val: 0.00000
Step:350, train: 0.17475-0.00000j, val: 0.00000
Step:351, train: 0.17121+0.00000j, val: 0.00000
Step:352, train: 0.16775+0.00000j, val: 0.00000
Step:353, train: 0.16435-0.00000j, val: 0.00000
Step:354, train: 0.16102+0.00000j, val: 0.00000
Step:355, train: 0.15776+0.00000j, val: 0.00000
Step:356, train: 0.15456-0.00000j, val: 0.00000
Step:357, train: 0.15143+0.00000j, val: 0.00000
Step:358, train: 0.14835-0.00000j, val: 0.00000
Step:359, train: 0.14534-0.00000j, val: 0.00000
Step:360, train: 0.14238+0.00000j, val: 

Step:512, train: 0.00505+0.00000j, val: 0.00000
Step:513, train: 0.00493+0.00000j, val: 0.00000
Step:514, train: 0.00481+0.00000j, val: 0.00000
Step:515, train: 0.00470+0.00000j, val: 0.00000
Step:516, train: 0.00459-0.00000j, val: 0.00000
Step:517, train: 0.00448-0.00000j, val: 0.00000
Step:518, train: 0.00438+0.00000j, val: 0.00000
Step:519, train: 0.00427+0.00000j, val: 0.00000
Step:520, train: 0.00417+0.00000j, val: 0.00000
Step:521, train: 0.00407-0.00000j, val: 0.00000
Step:522, train: 0.00398-0.00000j, val: 0.00000
Step:523, train: 0.00388+0.00000j, val: 0.00000
Step:524, train: 0.00379+0.00000j, val: 0.00000
Step:525, train: 0.00370-0.00000j, val: 0.00000
Step:526, train: 0.00361-0.00000j, val: 0.00000
Step:527, train: 0.00352+0.00000j, val: 0.00000
Step:528, train: 0.00344-0.00000j, val: 0.00000
Step:529, train: 0.00336+0.00000j, val: 0.00000
Step:530, train: 0.00328+0.00000j, val: 0.00000
Step:531, train: 0.00320+0.00000j, val: 0.00000
Step:532, train: 0.00312-0.00000j, val: 

...

Step:174, train: 0.00000+0.00000j, val: 0.00000
Step:175, train: 0.00000+0.00000j, val: 0.00000
Step:176, train: 0.00000+0.00000j, val: 0.00000
Step:177, train: 0.00000-0.00000j, val: 0.00000
Step:178, train: 0.00000+0.00000j, val: 0.00000
Step:179, train: 0.00000-0.00000j, val: 0.00000
Step:180, train: 0.00000-0.00000j, val: 0.00000
Step:181, train: 0.00000+0.00000j, val: 0.00000
Step:182, train: 0.00000-0.00000j, val: 0.00000
Step:183, train: 0.00000-0.00000j, val: 0.00000
Step:184, train: 0.00000-0.00000j, val: 0.00000
Step:185, train: 0.00000-0.00000j, val: 0.00000
Step:186, train: 0.00000+0.00000j, val: 0.00000
Step:187, train: 0.00000-0.00000j, val: 0.00000
Step:188, train: 0.00000+0.00000j, val: 0.00000
Step:189, train: 0.00000-0.00000j, val: 0.00000
Step:190, train: 0.00000-0.00000j, val: 0.00000
Step:191, train: 0.00000-0.00000j, val: 0.00000
Step:192, train: 0.00000+0.00000j, val: 0.00000
Step:193, train: 0.00000+0.00000j, val: 0.00000
Step:194, train: 0.00000+0.00000j, val: 

Step:346, train: 0.00000-0.00000j, val: 0.00000
Step:347, train: 0.00000+0.00000j, val: 0.00000
Step:348, train: 0.00000+0.00000j, val: 0.00000
Step:349, train: 0.00000-0.00000j, val: 0.00000
Step:350, train: 0.00000-0.00000j, val: 0.00000
Step:351, train: 0.00000-0.00000j, val: 0.00000
Step:352, train: 0.00000+0.00000j, val: 0.00000
Step:353, train: 0.00000-0.00000j, val: 0.00000
Step:354, train: 0.00000-0.00000j, val: 0.00000
Step:355, train: 0.00000+0.00000j, val: 0.00000
Step:356, train: 0.00000+0.00000j, val: 0.00000
Step:357, train: 0.00000-0.00000j, val: 0.00000
Step:358, train: 0.00000+0.00000j, val: 0.00000
Step:359, train: 0.00000-0.00000j, val: 0.00000
Step:360, train: 0.00000+0.00000j, val: 0.00000
Step:361, train: 0.00000-0.00000j, val: 0.00000
Step:362, train: 0.00000+0.00000j, val: 0.00000
Step:363, train: 0.00000-0.00000j, val: 0.00000
Step:364, train: 0.00000+0.00000j, val: 0.00000
Step:365, train: 0.00000+0.00000j, val: 0.00000
Step:366, train: 0.00000+0.00000j, val: 

Step:518, train: 0.00000-0.00000j, val: 0.00000
Step:519, train: 0.00000+0.00000j, val: 0.00000
Step:520, train: 0.00000-0.00000j, val: 0.00000
Step:521, train: 0.00000+0.00000j, val: 0.00000
Step:522, train: 0.00000+0.00000j, val: 0.00000
Step:523, train: 0.00000+0.00000j, val: 0.00000
Step:524, train: 0.00000-0.00000j, val: 0.00000
Step:525, train: 0.00000+0.00000j, val: 0.00000
Step:526, train: 0.00000-0.00000j, val: 0.00000
Step:527, train: 0.00000+0.00000j, val: 0.00000
Step:528, train: 0.00000-0.00000j, val: 0.00000
Step:529, train: 0.00000+0.00000j, val: 0.00000
Step:530, train: 0.00000-0.00000j, val: 0.00000
Step:531, train: 0.00000-0.00000j, val: 0.00000
Step:532, train: 0.00000+0.00000j, val: 0.00000
Step:533, train: 0.00000+0.00000j, val: 0.00000
Step:534, train: 0.00000+0.00000j, val: 0.00000
Step:535, train: 0.00000+0.00000j, val: 0.00000
Step:536, train: 0.00000+0.00000j, val: 0.00000
Step:537, train: 0.00000+0.00000j, val: 0.00000
Step:538, train: 0.00000+0.00000j, val: 

Step:690, train: 0.00000+0.00000j, val: 0.00000
Step:691, train: 0.00000+0.00000j, val: 0.00000
Step:692, train: 0.00000+0.00000j, val: 0.00000
Step:693, train: 0.00000+0.00000j, val: 0.00000
Step:694, train: 0.00000+0.00000j, val: 0.00000
Step:695, train: 0.00000+0.00000j, val: 0.00000
Step:696, train: -0.00000-0.00000j, val: 0.00000
Step:697, train: 0.00000-0.00000j, val: 0.00000
Step:698, train: 0.00000+0.00000j, val: 0.00000
Step:699, train: 0.00000+0.00000j, val: 0.00000
Step:700, train: 0.00000-0.00000j, val: 0.00000
Step:701, train: 0.00000-0.00000j, val: 0.00000
Step:702, train: 0.00000-0.00000j, val: 0.00000
Step:703, train: 0.00000+0.00000j, val: 0.00000
Step:704, train: 0.00000-0.00000j, val: 0.00000
Step:705, train: 0.00000-0.00000j, val: 0.00000
Step:706, train: 0.00000+0.00000j, val: 0.00000
Step:707, train: 0.00000+0.00000j, val: 0.00000
Step:708, train: 0.00000-0.00000j, val: 0.00000
Step:709, train: 0.00000+0.00000j, val: 0.00000
Step:710, train: 0.00000+0.00000j, val:

Step:862, train: 0.00000+0.00000j, val: 0.00000
Step:863, train: -0.00000-0.00000j, val: 0.00000
Step:864, train: -0.00000-0.00000j, val: 0.00000
Step:865, train: -0.00000-0.00000j, val: 0.00000
Step:866, train: 0.00000-0.00000j, val: 0.00000
Step:867, train: 0.00000+0.00000j, val: 0.00000
Step:868, train: 0.00000+0.00000j, val: 0.00000
Step:869, train: 0.00000-0.00000j, val: 0.00000
Step:870, train: 0.00000+0.00000j, val: 0.00000
Step:871, train: -0.00000-0.00000j, val: 0.00000
Step:872, train: -0.00000-0.00000j, val: 0.00000
Step:873, train: -0.00000-0.00000j, val: 0.00000
Step:874, train: 0.00000+0.00000j, val: 0.00000
Step:875, train: 0.00000+0.00000j, val: 0.00000
Step:876, train: 0.00000-0.00000j, val: 0.00000
Step:877, train: 0.00000-0.00000j, val: 0.00000
Step:878, train: -0.00000-0.00000j, val: 0.00000
Step:879, train: -0.00000-0.00000j, val: 0.00000
Step:880, train: 0.00000+0.00000j, val: 0.00000
Step:881, train: 0.00000+0.00000j, val: 0.00000
Step:882, train: 0.00000+0.00000

In [1]:
print(effective_rank(kraus1), effective_rank(kraus2))
print(attraction(kraus1), attraction(kraus2))

NameError: name 'effective_rank' is not defined